# Regridding Class Tmax/Tmin (focusing on feb AR case for now)

In [1]:
# imports
import numpy as np
import xarray as xr
import xesmf as xe
import Nio

In [2]:
# day strs for AR Feb case (temporary)
feb_AR_days = ['d02_1996-12-26','d02_1996-12-27','d02_1996-12-28','d02_1996-12-29','d02_1996-12-30',
               'd02_1996-12-31','d02_1997-01-01','d02_1997-01-02','d02_1997-01-03','d02_1997-01-04']
feb_AR_days_next = ['d02_1996-12-27','d02_1996-12-28','d02_1996-12-29','d02_1996-12-30','d02_1996-12-31',
                    'd02_1997-01-01','d02_1997-01-02','d02_1997-01-03','d02_1997-01-04','d02_1997-01-05']
feb_AR_days_prev = ['d02_1996-12-25','d02_1996-12-26','d02_1996-12-27','d02_1996-12-28','d02_1996-12-29',
                    'd02_1996-12-30','d02_1996-12-31','d02_1997-01-01','d02_1997-01-02','d02_1997-01-03']

# reading in helper files
fn_wrf = '/cw3e/mead/projects/cwp103/production/output/2017020500/cf/NG-GR-NQ-BD/wrfcf_d02_2017-02-06_00_00_00.nc'
fn2_liv = '/cw3e/mead/projects/cwp103/scratch/ldehaan/Livneh/ptt_2017.nc'

# regridder
da_input = xr.open_dataset(fn_wrf)
da_input2 = xr.open_dataset(fn2_liv)
regridder = xe.Regridder(da_input, da_input2, 'bilinear')


# class for regridding all wrf files
class regrid:
    
    # constructor initializing each case, run, and tmax/tmin option
    def __init__(self, case, run, maxmin):
        
        # input verification
        if isinstance(case, str) == False:
            raise Exception('Case definition is not a string.')
        elif isinstance(run, str) == False:
            raise Exception('Run definition is not a string.')
        elif maxmin not in ['max','min']:
            raise Exception('Max or Min was not defined.')
        
        # attribute constructor
        self.case = case
        self.run = run
        self.maxmin = maxmin
    
    # method regridding next day timing option, outputs netCDF
    def regrid_next(self):
        
        # regridding for each day
        for i in np.arange(len(feb_AR_days)):
            print(feb_AR_days[i] + ',' + self.case + ',' + self.run + ',' + self.maxmin + ',' + 'next')
            
            # helper function finding max/min per pt per day
            def value_per_pt(day, day_next):
                def_hrs1 = ['08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
                def_hrs2 = ['00','01','02','03','04','05','06','07']
                nested = []
                for i in def_hrs1:
                    fn = '/cw3e/mead/projects/cwp103/production/output/'+ self.case +'/cf/'+ self.run +'/wrfcf_'+ day +'_' + i + '_00_00.nc'
                    cdf = Nio.open_file(fn, "r")
                    arr = cdf.variables["T_2m"][:].flatten()
                    nested.append(arr)
                for j in def_hrs2:
                    fn2 = '/cw3e/mead/projects/cwp103/production/output/'+ self.case +'/cf/'+ self.run +'/wrfcf_'+ day_next +'_' + j + '_00_00.nc'
                    cdf2 = Nio.open_file(fn2, "r")
                    arr2 = cdf2.variables["T_2m"][:].flatten()
                    nested.append(arr2)
                ln = len(nested[0])
                lst = []
                for i in np.arange(ln):
                    if self.maxmin == 'max':
                        lst.append(max([item[i] for item in nested]))
                    elif self.maxmin == 'min':
                        lst.append(min([item[i] for item in nested]))
                return lst
            
            day_arr = value_per_pt(feb_AR_days[i],feb_AR_days_next[i])
            da_input['new'] = da_input['T_2m']
            da_input['new'].data[0] = np.array_split(day_arr, 1230)
            
            # regridding step
            dr_out = regridder(da_input['new'])
            dr_out[0].to_netcdf('./wrf_'+self.case+'_'+self.run+'_'+self.maxmin+feb_AR_days[i]+'_next.nc')
            print('File Completed')
            
    # method regridding prev day timing option, outputs netCDF
    def regrid_prev(self):
        
        # regridding for each day
        for i in np.arange(len(feb_AR_days)):
            print(feb_AR_days[i] + ',' + self.case + ',' + self.run + ',' + self.maxmin + ',' + 'prev')
            
            # helper function finding max/min per pt per day
            def value_per_pt2(day_prev, day):
                def_hrs1 = ['08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
                def_hrs2 = ['00','01','02','03','04','05','06','07']
                nested = []
                for i in def_hrs1:
                    fn = '/cw3e/mead/projects/cwp103/production/output/'+ self.case +'/cf/'+ self.run +'/wrfcf_'+ day_prev +'_' + i + '_00_00.nc'
                    cdf = Nio.open_file(fn, "r")
                    arr = cdf.variables["T_2m"][:].flatten()
                    nested.append(arr)
                for j in def_hrs2:
                    fn2 = '/cw3e/mead/projects/cwp103/production/output/'+ self.case +'/cf/'+ self.run +'/wrfcf_'+ day +'_' + j + '_00_00.nc'
                    cdf2 = Nio.open_file(fn2, "r")
                    arr2 = cdf2.variables["T_2m"][:].flatten()
                    nested.append(arr2)
                ln = len(nested[0])
                lst = []
                for i in np.arange(ln):
                    if self.maxmin == 'max':
                        lst.append(max([item[i] for item in nested]))
                    elif self.maxmin == 'min':
                        lst.append(min([item[i] for item in nested]))
                return lst
            
            day_arr = value_per_pt2(feb_AR_days_prev[i],feb_AR_days[i])
            da_input['new'] = da_input['T_2m']
            da_input['new'].data[0] = np.array_split(day_arr, 1230)
            
            # regridding step
            dr_out = regridder(da_input['new'])
            dr_out[0].to_netcdf('./wrf_'+self.case+'_'+self.run+'_'+self.maxmin+feb_AR_days[i]+'_prev.nc')
            print('File Completed')

In [3]:
#'NG-GR-YQ-BD','NG-SP-NQ-PO','OO-6D','OO-CO','OO-SC','PH-MO-GF-MY','PH-MO-TK-MY','PH-TH-GF-MY',
#runs = ['PH-TH-TK-MY','VL-100','VL-120','NG-GR-NQ-PO','NG-GR-YQ-PO','NG-SP-YQ-PO','OO-CC',
      #'NG-GR-NQ-BD','OO-NP','OO-VC','PH-MO-GF-YU','PH-MO-TK-YU','PH-TH-GF-YU','PH-TH-TK-YU','VL-110','VL-130']
runs = ['CC-TA-GF-MY'] #'CC-TA-GF-MY', 'CC-TH-GF-M3', 'CC-TH-GF-MY', 'NC-TH-GF-MY'
for x in runs:
    temp = regrid('1996122500',x,'min')
    temp.regrid_next()
    temp.regrid_prev()

d02_1996-12-26,1996122500,CC-TA-GF-MY,min,next


/cw3e/mead/projects/cwp103/scratch/eudoraf12/ncl_wrapper_environment/lib/python3.8/site-packages/xesmf/frontend.py:466: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


File Completed
d02_1996-12-27,1996122500,CC-TA-GF-MY,min,next
File Completed
d02_1996-12-28,1996122500,CC-TA-GF-MY,min,next
File Completed
d02_1996-12-29,1996122500,CC-TA-GF-MY,min,next
File Completed
d02_1996-12-30,1996122500,CC-TA-GF-MY,min,next
File Completed
d02_1996-12-31,1996122500,CC-TA-GF-MY,min,next
File Completed
d02_1997-01-01,1996122500,CC-TA-GF-MY,min,next
File Completed
d02_1997-01-02,1996122500,CC-TA-GF-MY,min,next


NIOError: Unable to open file

/cw3e/mead/projects/cwp103/production/output/2006072000/cf/NC-TH-GF-MY

In [ ]:
/cw3e/mead/projects/cwp103/production/output/1996122500/cf/CC-TA-GF-MY